# Radial Basis Function Neural Network

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [3]:
from itertools import product

import numpy as np
import xarray as xr
import xarray.ufuncs as xf
import matplotlib.pyplot as plt

In [4]:
a = np.array([[1, 2], [3, 4], [5, 6]])

In [7]:
def foo(x):
    print(f"{x=}")
    return x * 2

In [8]:
list(map(foo, a))

x=array([1, 2])
x=array([3, 4])
x=array([5, 6])


[array([2, 4]), array([6, 8]), array([10, 12])]